<a href="https://colab.research.google.com/github/lanej5/mlb/blob/main/elo/features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate features

The primary output of this notebook is `features.parquet`. Each row corresponds to a game. There are two labels:

- `RD` - the run differential of a game (scalar label)
- `W` - a binary label (1 if home team won, 0 if visitors won)

### todo

features to add:
- travel distance
- separate home / road stats ?

other things to try:
- other types of window

## References:
- https://www.mlb.com/stats/team/batting-average
- https://www.mlb.com/stats/team/pitching?sortState=asc
- https://library.fangraphs.com/pitching/babip/


In [1]:
import pandas as pd
pd.options.display.max_columns = None

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/mlb/retrosheet/'
! ls '/content/drive/My Drive/mlb/retrosheet/'

biofile.csv  GL1995.csv  GL2001.csv  GL2007.csv  GL2013.csv  GL2019.csv
GL1990.csv   GL1996.csv  GL2002.csv  GL2008.csv  GL2014.csv  GL2020.csv
GL1991.csv   GL1997.csv  GL2003.csv  GL2009.csv  GL2015.csv  GL2021.csv
GL1992.csv   GL1998.csv  GL2004.csv  GL2010.csv  GL2016.csv  LEAGUEABR.csv
GL1993.csv   GL1999.csv  GL2005.csv  GL2011.csv  GL2017.csv  parks.csv
GL1994.csv   GL2000.csv  GL2006.csv  GL2012.csv  GL2018.csv  TEAMABR.csv


# legend

In [4]:
!wget https://www.retrosheet.org/gamelogs/glfields.txt

!cat glfields.txt

--2022-07-08 19:55:54--  https://www.retrosheet.org/gamelogs/glfields.txt
Resolving www.retrosheet.org (www.retrosheet.org)... 192.124.249.9
Connecting to www.retrosheet.org (www.retrosheet.org)|192.124.249.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6559 (6.4K) [text/plain]
Saving to: ‘glfields.txt’

glfields.txt        100%[===================>]   6.41K  --.-KB/s    in 0s      

2022-07-08 19:55:54 (566 MB/s) - ‘glfields.txt’ saved [6559/6559]

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B"

# load, clean, concat game logs 1990 - 2021

In [5]:
import pandas as pd
from typing import Dict

def get_team_index_map(file: str) -> Dict['str', int]:
    """Returns a dict mapping team abbr from 1990 - 2021 to a
    dense set of indices.

    If a team changed at some point between 1990 - 2021, the old
    team abbr is mapped to the same index as the new team abbr.

    Args:
    -----
    file: valid path to TEAMABR.csv from retrosheets.
    """
    teams = pd.read_csv(file)

    # filter out teams that ceased to exist before 2021
    current_teams = teams[teams['Last year of this combination'] == 2021] \
    .reset_index(drop=True)

    # create mapping
    team_index_map = dict(zip(current_teams['Team Abbreviation'], current_teams.index))

    # add changed team abbr
    team_index_map['MON'] = team_index_map['WAS']
    team_index_map['CAL'] = team_index_map['ANA']
    team_index_map['FLO'] = team_index_map['MIA']

    # save as a dataframe
    team_ids = pd.DataFrame(team_index_map.items(), columns=['abbr', 'id'])
    team_ids.to_csv('/content/drive/My Drive/mlb/features/team_ids.csv')

    return team_index_map

def clean(gl):
    """
    clean the dataframe:
        1. parse dates
        2. convert abbr to team ids 
        3. rename stat columns 
        4. add run differential and win columns
        5. drop tie games
        6. add integer date index 
        7. sort dates
        8. add game index
    """
    team_ids = get_team_index_map('/content/drive/My Drive/mlb/retrosheet/TEAMABR.csv')

    # dict for renaming stat columns
    cols = {
        '4': 'visitor_L',
        '7': 'home_L',
        '9': 'visitor_R',
        '10': 'home_R',
        '16': 'park_ID',
        '21': 'visitor_AB',
        '22': 'visitor_H',
        '23': 'visitor_2B',
        '24': 'visitor_3B',
        '25': 'visitor_HR',
        '28': 'visitor_SF',
        '29': 'visitor_HBP',
        '30': 'visitor_BB',
        '40': 'visitor_ER',
        '49': 'home_AB',
        '50': 'home_H',
        '51': 'home_2B',
        '52': 'home_3B',
        '53': 'home_HR',
        '56': 'home_SF',
        '57': 'home_HBP',
        '58': 'home_BB',
        '68': 'home_ER'
    }

    # parse dates
    gl = gl.assign(date=pd.to_datetime(gl['0'], format='%Y%m%d'))
    gl = gl.assign(season=gl['date'].dt.year)

    # replace abbr with team_id
    gl = gl.assign(visitor_id=gl['3'].map(team_ids)) \
    .assign(home_id=gl['6'].map(team_ids))

    # rename columns
    gl = gl.rename(columns=cols)

    # drop unnamed columns
    to_drop = list(set(gl.columns).intersection([str(i) for i in range(161)]))
    gl = gl.drop(columns=to_drop)
    
    # assign
    gl = gl.assign(home_1B=gl['home_H'] - gl['home_2B'] - gl['home_3B'] - gl['home_HR'])
    gl = gl.assign(visitor_1B=gl['visitor_H'] - gl['visitor_2B'] - gl['visitor_3B'] - gl['visitor_HR'])
    gl = gl.assign(RD=gl['home_R'] - gl['visitor_R'])
    gl = gl.assign(W=(gl['RD'] > 0).astype(int))
    gl = gl.assign(interleague=(gl['home_L'] != gl['visitor_L']).astype(int))

    # drop tie games
    gl = gl.drop(gl[gl['RD'] == 0].index)

    # add date and game indexes
    gl = gl.assign(date_index=gl['date'].rank(method='dense'))
    gl = gl.sort_values('date')
    gl = gl.reset_index(drop=True) \
    .reset_index(drop=False).rename(columns={'index': 'game_id'})

    return gl

def load_game_logs():
    gls = []

    for y in range(1990, 2022):
        filename = '/content/drive/My Drive/mlb/retrosheet/GL' + str(y) + '.csv'
        gls.append(pd.read_csv(filename))

    return pd.concat(gls).pipe(clean)

In [6]:
# create game logs
gl = load_game_logs()
gl.head()

game_id visitor_L home_L  visitor_R  home_R park_ID  visitor_AB  visitor_H  \
0        0        AL     AL          2       5   BOS07          33          5   
1        1        AL     AL          7       4   ANA01          40         15   
2        2        AL     AL          1       2   CHI10          30          4   
3        3        AL     AL          3       8   OAK01          37         10   
4        4        AL     AL          2       4   ARL01          28          3   

   visitor_2B  visitor_3B  visitor_HR  visitor_SF  visitor_HBP  visitor_BB  \
0           1           0           0           1            0           4   
1           2           0           1           0            0           1   
2           1           0           0           0            1           1   
3           1           0           1           0            0           2   
4           2           0           0           0            0           5   

   visitor_ER  home_AB  home_H  home_2B  home_3B  home_HR  home_SF  home_HBP  \
0           2       33       8        4        0        0        0         0   
1           2       31       5        0        0        1        1         0   
2           2       29       6        1        1        0        1         0   
3           7       32      11        2        0        2        1         0   
4           4       30       7        1        1        2        0         1   

   home_BB  home_ER       date  season  visitor_id  home_id  home_1B  \
0        4        2 1990-04-09    1990           5        2        4   
1        5        7 1990-04-09    1990          11        0        4   
2        2        1 1990-04-09    1990          23        3        4   
3        6        3 1990-04-09    1990           8       10        7   
4        6        2 1990-04-09    1990          14       13        3   

   visitor_1B  RD  W  interleague  date_index  
0           4   3  1            0         1.0  
1          12  -3  0            0         1.0  
2           3   1  1            0         1.0  
3           8   5  1            0         1.0  
4           1   2  1            0         1.0

# ballparks and travel distance

Travel distance of visiting team as a feature.

In [73]:
from datetime import datetime

!wget https://www.retrosheet.org/parkcode.txt

parkcode = pd.read_csv('parkcode.txt', parse_dates=['START', 'END'])

# select only parks used since 1990
parkcode = parkcode[(parkcode['END'] > datetime(1990, 1, 1)) ^ parkcode['END'].isna()]

# drop outliers (both teams need to travel to these remote locations so there's no travel advantage anyways)
parkcode = parkcode[~parkcode['PARKID'].isin(['HON01', 'LON01', 'TOK01', 'SYD01', 'SJU01', 'MNT01'])]

parkcode

--2022-07-08 21:00:53--  https://www.retrosheet.org/parkcode.txt
Resolving www.retrosheet.org (www.retrosheet.org)... 192.124.249.9
Connecting to www.retrosheet.org (www.retrosheet.org)|192.124.249.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20707 (20K) [text/plain]
Saving to: ‘parkcode.txt.19’

parkcode.txt.19     100%[===================>]  20.22K  --.-KB/s    in 0.02s   

2022-07-08 21:00:53 (1.18 MB/s) - ‘parkcode.txt.19’ saved [20707/20707]



PARKID                           NAME  \
2    ANA01       Angel Stadium of Anaheim   
3    ARL01              Arlington Stadium   
4    ARL02  Rangers Ballpark in Arlington   
5    ARL03  Globe Life Field in Arlington   
6    ATL01  Atlanta-Fulton County Stadium   
..     ...                            ...   
223  STP01                Tropicana Field   
235  TOR02                  Rogers Centre   
248  WAS10      Robert F. Kennedy Stadium   
249  WAS11                 Nationals Park   
255  WIL02  BB&T Ballpark at Bowman Field   

                                          AKA            CITY STATE  \
2               Edison Field; Anaheim Stadium         Anaheim    CA   
3                                         NaN       Arlington    TX   
4    The Ballpark in Arlington; Ameriquest Fl       Arlington    TX   
5                                         NaN       Arlington    TX   
6                                         NaN         Atlanta    GA   
..                                        ...             ...   ...   
223                                       NaN  St. Petersburg    FL   
235                                   Skydome         Toronto   ONT   
248                              D.C. Stadium      Washington    DC   
249                                       NaN      Washington    DC   
255                                       NaN    Williamsport    PA   

         START        END LEAGUE                   NOTES  
2   1966-04-19        NaT     AL                     NaN  
3   1972-04-21 1993-10-03     AL                     NaN  
4   1994-04-11 2019-09-29     AL                     NaN  
5   2020-07-24        NaT     AL                     NaN  
6   1966-04-12 1996-09-23     NL                     NaN  
..         ...        ...    ...                     ...  
223 1998-03-31        NaT     AL                     NaN  
235 1989-06-05        NaT     AL                     NaN  
248 1962-04-09 2007-09-23    NaN  AL:1962-71; NL:2005-07  
249 2008-03-30        NaT     NL                     NaN  
255 2017-08-20 2017-08-20     NL                     PIT  

[62 rows x 9 columns]

In [77]:
# some parks have the same location (city, state)
# let's use unique city, state pairs to get geolocation data 
# because this is rate limited and slow

locations = parkcode[['CITY', 'STATE']].drop_duplicates()
len(locations)

37

In [25]:
! pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [78]:
# get latitude and longitude of parks 
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="mlb_postseason_dash")

lat_long = []

for i, r in locations.iterrows():
    location_name = r['CITY'] + ", " + r['STATE']
    location = geolocator.geocode(location_name)
    lat_long.append((r['CITY'], r['STATE'], location.latitude, location.longitude))
    time.sleep(2) # avoid getting rate limited by openstreetmap

lat_long_df = pd.DataFrame(data=lat_long, columns=['CITY', 'STATE', 'lat', 'long'])

In [81]:
lat_long_df

CITY STATE        lat        long
0            Anaheim    CA  33.834752 -117.911732
1          Arlington    TX  32.735582  -97.107119
2            Atlanta    GA  33.748992  -84.390264
3          Baltimore    MD  39.290882  -76.610759
4             Boston    MA  42.355433  -71.060511
5            Buffalo    NY  42.886717  -78.878392
6            Chicago    IL  41.875562  -87.624421
7         Cincinnati    OH  39.101454  -84.512460
8          Cleveland    OH  41.499656  -81.693681
9             Denver    CO  39.739236 -104.984862
10           Detroit    MI  42.331551  -83.046640
11           Dunedin    FL  28.019740  -82.771768
12        Dyersville    IA  42.484460  -91.121106
13        Fort Bragg    NC  35.140311  -79.154146
14           Houston    TX  29.758938  -95.367697
15       Kansas City    MO  39.100105  -94.578142
16         Las Vegas    NV  36.167256 -115.148516
17  Lake Buena Vista    FL  28.380350  -81.529758
18       Los Angeles    CA  34.053691 -118.242766
19             Miami    FL  25.774173  -80.193620
20         Milwaukee    WI  43.034993  -87.922497
21       Minneapolis    MN  44.977300  -93.265469
22          Montreal   QUE  45.453601  -73.641879
23          New York    NY  40.712728  -74.006015
24           Oakland    CA  37.804456 -122.271356
25             Omaha    NE  41.258746  -95.938376
26      Philadelphia    PA  39.952724  -75.163526
27           Phoenix    AZ  33.448437 -112.074141
28        Pittsburgh    PA  40.441694  -79.990086
29         San Diego    CA  32.717420 -117.162773
30           Seattle    WA  47.603832 -122.330062
31     San Francisco    CA  37.779026 -122.419906
32         St. Louis    MO  38.626426  -90.199585
33    St. Petersburg    FL  27.770380  -82.669509
34           Toronto   ONT  43.561411  -79.702717
35        Washington    DC  38.895037  -77.036543
36      Williamsport    PA  41.249329  -77.002767

In [ ]:
# get pairwise distances between cities

from geopy.distance import geodesic

distances = []

for i in range(len(lat_long_df)):
    c_i, s_i = lat_long_df.loc[i, 'CITY'], lat_long_df.loc[i, 'STATE']
    ll_i = (lat_long_df.loc[i, 'lat'], lat_long_df.loc[i, 'long'])
    for j in range(i+1, len(lat_long_df)):
        ll_j = (lat_long_df.loc[j, 'lat'], lat_long_df.loc[j, 'long'])
        d = geodesic(ll_i, ll_j).kilometers
        distances.append((c_i, s_i, lat_long_df.loc[j, 'CITY'], lat_long_df.loc[j, 'STATE'], d))
        time.sleep(1.1) # avoid getting rate limited by openstreetmap



In [85]:
distances_df = pd.DataFrame(data=distances, columns=['CITY_1', 'STATE_1', 'CITY_2', 'STATE_2', 'geodesic distance (km)'])

In [86]:
distances_df

CITY_1 STATE_1        CITY_2 STATE_2  geodesic distance (km)
0           Anaheim      CA     Arlington      TX             1938.513119
1           Anaheim      CA       Atlanta      GA             3090.411121
2           Anaheim      CA     Baltimore      MD             3713.824667
3           Anaheim      CA        Boston      MA             4162.349483
4           Anaheim      CA       Buffalo      NY             3521.477167
..              ...     ...           ...     ...                     ...
661  St. Petersburg      FL    Washington      DC             1339.915504
662  St. Petersburg      FL  Williamsport      PA             1582.247817
663         Toronto     ONT    Washington      DC              564.312929
664         Toronto     ONT  Williamsport      PA              339.611785
665      Washington      DC  Williamsport      PA              261.427195

[666 rows x 5 columns]

In [110]:
dist_df = pd.merge(
    parkcode[['PARKID', 'CITY', 'STATE']],
    distances_df,
    how='left',
    left_on=['CITY', 'STATE'],
    right_on=['CITY_1', 'STATE_1']
).drop(columns=['CITY', 'STATE', 'CITY_1', 'STATE_1']).dropna()

dist_df = pd.merge(
    parkcode[['PARKID', 'CITY', 'STATE']],
    dist_df,
    how='left',
    left_on=['CITY', 'STATE'],
    right_on=['CITY_2', 'STATE_2']
).drop(columns=['CITY', 'STATE', 'CITY_2', 'STATE_2']).dropna()

dist_df

PARKID_x PARKID_y  geodesic distance (km)
1       ARL01    ANA01             1938.513119
2       ARL02    ANA01             1938.513119
3       ARL03    ANA01             1938.513119
4       ATL01    ANA01             3090.411121
5       ATL01    ARL01             1189.724279
...       ...      ...                     ...
1856    WIL02    STL10             1163.643877
1857    WIL02    STP01             1582.247817
1858    WIL02    TOR02              339.611785
1859    WIL02    WAS10              261.427195
1860    WIL02    WAS11              261.427195

[1860 rows x 3 columns]

In [111]:
dist_df_mirror = dist_df.rename(columns={'PARKID_x': 'PARKID_y', 'PARKID_y': 'PARKID_x'})

dist_df = pd.concat([dist_df, dist_df_mirror], axis=0)
dist_df

PARKID_x PARKID_y  geodesic distance (km)
1       ARL01    ANA01             1938.513119
2       ARL02    ANA01             1938.513119
3       ARL03    ANA01             1938.513119
4       ATL01    ANA01             3090.411121
5       ATL01    ARL01             1189.724279
...       ...      ...                     ...
1856    STL10    WIL02             1163.643877
1857    STP01    WIL02             1582.247817
1858    TOR02    WIL02              339.611785
1859    WAS10    WIL02              261.427195
1860    WAS11    WIL02              261.427195

[3720 rows x 3 columns]

In [113]:
dist_df.duplicated().sum()

0

In [114]:
dist_df.to_csv("/content/drive/My Drive/mlb/features/park_distances.csv", index=False)

# create team log

In [170]:
def get_team_logs(gl: pd.DataFrame) -> pd.DataFrame:
    """Create team logs from game logs."""

    # columns to use
    cols = [
        'game_id',
        'RD',
        'W',
        'date',
        'season'
    ] + [c for c in gl.columns if c.startswith(('home', 'visitor'))]

    # home logs
    h_rename = {
        **{c: 'team' + c[4:] for c in gl.columns if c.startswith('home')},
        **{c: 'opp' + c[7:] for c in gl.columns if c.startswith('visitor')}
    }

    hl = gl[cols].rename(columns=h_rename)

    # visitor logs
    v_rename = {
        **{c: 'opp' + c[4:] for c in gl.columns if c.startswith('home')},
        **{c: 'team' + c[7:] for c in gl.columns if c.startswith('visitor')}
    }

    vl = gl[cols].rename(columns=v_rename)

    vl['RD'] = - vl['RD']
    vl['W'] = 1 - vl['W']

    tl = pd.concat([hl, vl], axis=0) \
        .reset_index(drop=True) \
        .sort_values(['team_id', 'game_id'])

    return tl

tl = get_team_logs(gl)
tl.head(15)

game_id  RD  W       date  season opp_L team_L  opp_R  team_R  opp_AB  \
1            1  -3  0 1990-04-09    1990    AL     AL      7       4      40   
15          15   7  1 1990-04-10    1990    AL     AL      0       7      30   
25          25   1  1 1990-04-11    1990    AL     AL      0       1      25   
35          35   2  1 1990-04-12    1990    AL     AL      1       3      29   
47          47  -3  0 1990-04-13    1990    AL     AL      7       4      33   
58          58   2  1 1990-04-14    1990    AL     AL      5       7      45   
72          72   3  1 1990-04-15    1990    AL     AL      1       4      27   
94          94  -2  0 1990-04-17    1990    AL     AL      7       5      45   
107        107  -2  0 1990-04-18    1990    AL     AL      3       1      33   
73260      124 -12  0 1990-04-20    1990    AL     AL     13       1      32   
73272      136  -8  0 1990-04-21    1990    AL     AL      8       0      36   
73283      147   3  1 1990-04-22    1990    AL     AL      2       5      31   
73305      169  -2  0 1990-04-24    1990    AL     AL      4       2      32   
73319      183   2  1 1990-04-25    1990    AL     AL      1       3      38   
73329      193  -1  0 1990-04-26    1990    AL     AL      5       4      31   

       opp_H  opp_2B  opp_3B  opp_HR  opp_SF  opp_HBP  opp_BB  opp_ER  \
1         15       2       0       1       0        0       1       2   
15         4       2       0       0       0        0       1       7   
25         0       0       0       0       0        0       4       1   
35         6       1       0       0       1        0       6       0   
47        11       3       0       0       2        1       6       3   
58        12       3       1       1       0        0       5       7   
72         3       1       0       0       0        1       3       2   
94        12       2       0       2       0        0       3       5   
107        7       0       0       1       0        0       8       1   
73260     11       2       0       2       2        1      10       0   
73272     12       3       0       2       0        1       6       0   
73283      8       3       0       0       0        0       1       5   
73305      8       3       0       0       0        0       4       2   
73319     13       0       0       1       0        0       4       3   
73329     11       1       0       1       1        0       3       4   

       team_AB  team_H  team_2B  team_3B  team_HR  team_SF  team_HBP  team_BB  \
1           31       5        0        0        1        1         0        5   
15          36      12        1        0        3        1         0        4   
25          30       7        3        0        0        0         0        6   
35          30       6        1        0        0        0         0        0   
47          35       9        2        0        1        0         0        2   
58          44      11        3        0        3        0         0        5   
72          27       4        1        0        0        0         1        3   
94          41       5        1        0        2        0         0        3   
107         35      10        1        0        0        0         0        2   
73260       34       6        2        0        0        0         0        0   
73272       31       6        2        0        0        0         2        5   
73283       35      10        4        0        0        0         0        5   
73305       33       6        1        0        0        0         0        1   
73319       35       9        3        0        0        0         0        1   
73329       33      10        2        0        1        0         0        5   

       team_ER  opp_id  team_id  team_1B  opp_1B  
1            7      11        0        4      12  
15           0      11        0        8       2  
25           0      11        0        4       0  
35           1       8        0        5       5  
47   

# add time series features

### Rest factor

Rest factor is the proportion of the last n days that were rest days.

In [171]:
from typing import Callable
import unittest
        

def rest_aggregator(window_size: int) -> Callable[[pd.Series], int]:
    """Rest aggregator factory.
    
    Args:
    -----
    window_size: the number of days prior to the current day over
        which to compute the rest factor.

    Returns:
    --------
    An aggregator for the given window size.
    """
    def rest_factor(s: pd.Series):
        # return proportion of last window_size days
        # that were rest days
        gaps = 0
        total = 0
        for diff in s.tolist()[::-1]:

            if diff > 1:
                if total + diff <= window_size:
                    gaps += diff - 1
                else:
                    gaps += window_size - total
            
            total += diff
            if total >= window_size:
                break
            
        if total < window_size:
            gaps += window_size - total

        return gaps / window_size
    
    return rest_factor


class TestRestAggregator(unittest.TestCase):

    def test_one(self):
        rest = rest_aggregator(5)
        diffs = [4,3,2,1]
        s = pd.Series(data=diffs)
        result = rest(s)
        # print(result)
        assert rest(s) == .6

    def test_two(self):
        dates = pd.to_datetime(
            ['2022-01-01',
             '2022-01-02',
             '2022-01-04',
             '2022-01-05',
             '2022-01-05',
             '2022-01-06',
             '2022-01-08',
             '2022-01-10',
             '2022-03-01'
            ]
        )
        s = pd.Series(data=dates)

        rest = rest_aggregator(4)
        result = s.diff().dt.days.fillna(1000).rolling(10, min_periods=1).agg(rest)
        # print(result)
        assert (result == pd.Series(data=[1,.75,.5,.25,.25,.25,.25,.5,1])).all()
        

unittest.main(argv=[''], verbosity=2, exit=False)

test_one (__main__.TestRestAggregator) ... ok
test_two (__main__.TestRestAggregator) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.016s

OK


In [172]:
tl.groupby(by='team_id', as_index=False)['date'].diff().head(10)

date
1        NaT
15    1 days
25    1 days
35    1 days
47    1 days
58    1 days
72    1 days
94    2 days
107   1 days
73260 2 days

In [173]:
# test rest aggregator
rest = rest_aggregator(4)
result = tl.groupby(by='team_id', as_index=False)['date'].diff()['date'].dt.days.fillna(1000).rolling(8, min_periods=1).agg(rest)
# pd.concat([result, tl['date']], axis=1)
result.head(10)

1        1.00
15       0.75
25       0.50
35       0.25
47       0.00
58       0.00
72       0.00
94       0.25
107      0.25
73260    0.50
Name: date, dtype: float64

### Compute features

In [174]:
# create groupby object over last n games
tl_groupby = tl.groupby('team_id', as_index=False)

# run differential over last n games
rd_window_sizes = [10, 20, 30, 50, 100]
for i in rd_window_sizes:
    tl['RD_' + str(i)] = tl_groupby['RD'].shift().rolling(i).mean().bfill()

# win percentage over last n games
win_pct_window_sizes = [10, 20, 30, 50, 100]
for i in win_pct_window_sizes:
    tl['win_pct_' + str(i)] = tl_groupby['W'].shift().rolling(2 * i).mean().bfill()

# rest over last n games
rest_window_sizes = list(range(1, 15))
for i in rest_window_sizes:
    rest_agg = rest_aggregator(i)
    tl['rest_' + str(i)] = tl_groupby['date'] \
        .diff()['date'].dt.days \
        .fillna(1000) \
        .rolling(i, min_periods=1) \
        .agg(rest_agg)

# stats over last n games
avg_window_sizes = [5, 10, 20, 30, 50, 100]
for i in avg_window_sizes:
    # offensive stats
    off_h    = tl_groupby['team_H'].shift().rolling(i).sum()['team_H']
    off_ab   = tl_groupby['team_AB'].shift().rolling(i).sum()['team_AB']
    off_bb   = tl_groupby['team_BB'].shift().rolling(i).sum()['team_BB']
    off_hbp  = tl_groupby['team_HBP'].shift().rolling(i).sum()['team_HBP']
    off_sf   = tl_groupby['team_SF'].shift().rolling(i).sum()['team_SF']
    off_1b   = tl_groupby['team_1B'].shift().rolling(i).sum()['team_1B']
    off_2b   = tl_groupby['team_2B'].shift().rolling(i).sum()['team_2B']
    off_3b   = tl_groupby['team_3B'].shift().rolling(i).sum()['team_3B']
    off_hr   = tl_groupby['team_HR'].shift().rolling(i).sum()['team_HR']
    
    # avg = H / AB
    tl['off_avg_' + str(i)] = (off_h / off_ab).bfill() 
    # OBP = (H + BB + HBP) / (AB + BB + HBP + SF)
    tl['off_obp_' + str(i)] = ((off_h + off_bb + off_hbp) / (off_ab + off_bb + off_hbp + off_sf)).bfill()
    # SLG = (1B + 2*2B + 3*3B + 4*HR) / AB
    tl['off_slg_' + str(i)] = ((off_1b + 2 * off_2b + 3 * off_3b + 4 * off_hr) / off_ab).bfill()
    # OPS = OBP + SLG
    # tl['off_slg_' + str(i)] = (tl['off_obp_' + str(i)] + tl['off_slg_' + str(i)]).bfill()

    # defensive stats
    def_h = tl_groupby['opp_H'].shift().rolling(i).sum()['opp_H']
    def_ab = tl_groupby['opp_AB'].shift().rolling(i).sum()['opp_AB']
    def_bb = tl_groupby['opp_BB'].shift().rolling(i).sum()['opp_BB']
    def_ip = def_ab / 3.0
    def_er = tl_groupby['team_ER'].shift().rolling(i).sum()['team_ER']

    # avg = H / AB
    tl['def_avg_' + str(i)] = (def_h / def_ab).bfill()
    # WHIP = (W + H) / IP
    tl['def_whip_' + str(i)] = ((def_bb + def_h)/ def_ip).bfill()
    # ERA = 9 * ER / IP
    tl['def_era_' + str(i)] = (9 * def_er / def_ip).bfill()

tl.head(15)

game_id  RD  W       date  season opp_L team_L  opp_R  team_R  opp_AB  \
1            1  -3  0 1990-04-09    1990    AL     AL      7       4      40   
15          15   7  1 1990-04-10    1990    AL     AL      0       7      30   
25          25   1  1 1990-04-11    1990    AL     AL      0       1      25   
35          35   2  1 1990-04-12    1990    AL     AL      1       3      29   
47          47  -3  0 1990-04-13    1990    AL     AL      7       4      33   
58          58   2  1 1990-04-14    1990    AL     AL      5       7      45   
72          72   3  1 1990-04-15    1990    AL     AL      1       4      27   
94          94  -2  0 1990-04-17    1990    AL     AL      7       5      45   
107        107  -2  0 1990-04-18    1990    AL     AL      3       1      33   
73260      124 -12  0 1990-04-20    1990    AL     AL     13       1      32   
73272      136  -8  0 1990-04-21    1990    AL     AL      8       0      36   
73283      147   3  1 1990-04-22    1990    AL     AL      2       5      31   
73305      169  -2  0 1990-04-24    1990    AL     AL      4       2      32   
73319      183   2  1 1990-04-25    1990    AL     AL      1       3      38   
73329      193  -1  0 1990-04-26    1990    AL     AL      5       4      31   

       opp_H  opp_2B  opp_3B  opp_HR  opp_SF  opp_HBP  opp_BB  opp_ER  \
1         15       2       0       1       0        0       1       2   
15         4       2       0       0       0        0       1       7   
25         0       0       0       0       0        0       4       1   
35         6       1       0       0       1        0       6       0   
47        11       3       0       0       2        1       6       3   
58        12       3       1       1       0        0       5       7   
72         3       1       0       0       0        1       3       2   
94        12       2       0       2       0        0       3       5   
107        7       0       0       1       0        0       8       1   
73260     11       2       0       2       2        1      10       0   
73272     12       3       0       2       0        1       6       0   
73283      8       3       0       0       0        0       1       5   
73305      8       3       0       0       0        0       4       2   
73319     13       0       0       1       0        0       4       3   
73329     11       1       0       1       1        0       3       4   

       team_AB  team_H  team_2B  team_3B  team_HR  team_SF  team_HBP  team_BB  \
1           31       5        0        0        1        1         0        5   
15          36      12        1        0        3        1         0        4   
25          30       7        3        0        0        0         0        6   
35          30       6        1        0        0        0         0        0   
47          35       9        2        0        1        0         0        2   
58          44      11        3        0        3        0         0        5   
72          27       4        1        0        0        0         1        3   
94          41       5        1        0        2        0         0        3   
107         35      10        1        0        0        0         0        2   
73260       34       6        2        0        0        0         0        0   
73272       31       6        2        0        0        0         2        5   
73283       35      10        4        0        0        0         0        5   
73305       33       6        1        0        0        0         0        1   
73319       35       9        3        0        0        0         0        1   
73329       33      10        2        0        1        0         0        5   

       team_ER  opp_id  team_id  team_1B  opp_1B  RD_10  RD_20     RD_30  \
1            7      11        0        4      12   -0.7  -0.45 -0.933333   
15           0      11        0        8       2   -0.7  -0.45 -0.933333   
25           0      11        0 

In [175]:
tl[tl['game_id'] == 0]

game_id  RD  W       date  season opp_L team_L  opp_R  team_R  opp_AB  \
0            0   3  1 1990-04-09    1990    AL     AL      2       5      33   
73136        0  -3  0 1990-04-09    1990    AL     AL      5       2      33   

       opp_H  opp_2B  opp_3B  opp_HR  opp_SF  opp_HBP  opp_BB  opp_ER  \
0          5       1       0       0       1        0       4       2   
73136      8       4       0       0       0        0       4       2   

       team_AB  team_H  team_2B  team_3B  team_HR  team_SF  team_HBP  team_BB  \
0           33       8        4        0        0        0         0        4   
73136       33       5        1        0        0        1         0        4   

       team_ER  opp_id  team_id  team_1B  opp_1B  RD_10  RD_20  RD_30  RD_50  \
0            2       5        2        4       4   -2.3    0.2    0.0  -0.04   
73136        2       2        5        4       4    0.0   -0.9   -1.1  -0.40   

       RD_100  win_pct_10  win_pct_20  win_pct_30  win_pct_50  win_pct_100  \
0       -0.36         0.6       0.525        0.55        0.52        0.555   
73136   -0.14         0.4       0.450        0.45        0.48        0.490   

       rest_1  rest_2  rest_3  rest_4  rest_5  rest_6  rest_7  rest_8  rest_9  \
0         1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
73136     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   

       rest_10  rest_11  rest_12  rest_13  rest_14  off_avg_5  off_obp_5  \
0          1.0      1.0      1.0      1.0      1.0   0.322034   0.396040   
73136      1.0      1.0      1.0      1.0      1.0   0.263158   0.345178   

       off_slg_5  def_avg_5  def_whip_5  def_era_5  off_avg_10  off_obp_10  \
0       0.446328   0.257310    1.140351   3.157895    0.253049    0.338667   
73136   0.350877   0.301775    1.313609   3.514793    0.281899    0.358442   

       off_slg_10  def_avg_10  def_whip_10  def_era_10  off_avg_20  \
0        0.344512    0.284091     1.159091    3.988636    0.272727   
73136    0.385757    0.292035     1.247788    3.185841    0.272069   

       off_obp_20  off_slg_20  def_avg_20  def_whip_20  def_era_20  \
0        0.360208    0.385991    0.243004     1.011782    2.902798   
73136    0.344031    0.397974    0.297376     1.298834    3.896501   

       off_avg_30  off_obp_30  off_slg_30  def_avg_30  def_whip_30  \
0        0.265672    0.346526    0.378109    0.249008     1.056548   
73136    0.270192    0.330427    0.425962    0.283317     1.243830   

       def_era_30  off_avg_50  off_obp_50  off_slg_50  def_avg_50  \
0        3.026786    0.268524    0.346982    0.384707    0.258886   
73136    4.051333    0.265889    0.337312    0.418659    0.271598   

       def_whip_50  def_era_50  off_avg_100  off_obp_100  off_slg_100  \
0         1.059242    3.103081     0.268532     0.342600     0.387615   
73136     1.180473    3.642604     0.258667     0.336914     0.410667   

       def_avg_100  def_whip_100  def_era_100  
0         0.265841      1.071618     3.055703  
73136     0.256410      1.136852     3.485689

In [176]:
# drop columns from tl that aren't features
tl_drop_cols = ['RD', 'W', 'date', 'season'] + [c for c in tl.columns if c.startswith('team') and c != 'team_id']
tl_drop_cols += [c for c in tl.columns if c.startswith('opp')]
tl_features = tl.drop(columns=tl_drop_cols)

tl_features.head()

game_id  team_id  RD_10  RD_20     RD_30  RD_50  RD_100  win_pct_10  \
1         1        0   -0.7  -0.45 -0.933333  -0.42    -0.5        0.45   
15       15        0   -0.7  -0.45 -0.933333  -0.42    -0.5        0.45   
25       25        0   -0.7  -0.45 -0.933333  -0.42    -0.5        0.45   
35       35        0   -0.7  -0.45 -0.933333  -0.42    -0.5        0.45   
47       47        0   -0.7  -0.45 -0.933333  -0.42    -0.5        0.45   

    win_pct_20  win_pct_30  win_pct_50  win_pct_100  rest_1  rest_2    rest_3  \
1        0.425    0.516667        0.48        0.505     1.0     1.0  1.000000   
15       0.425    0.516667        0.48        0.505     0.0     0.5  0.666667   
25       0.425    0.516667        0.48        0.505     0.0     0.0  0.333333   
35       0.425    0.516667        0.48        0.505     0.0     0.0  0.000000   
47       0.425    0.516667        0.48        0.505     0.0     0.0  0.000000   

    rest_4  rest_5    rest_6    rest_7  rest_8    rest_9  rest_10   rest_11  \
1     1.00     1.0  1.000000  1.000000   1.000  1.000000      1.0  1.000000   
15    0.75     0.8  0.833333  0.857143   0.875  0.888889      0.9  0.909091   
25    0.50     0.6  0.666667  0.714286   0.750  0.777778      0.8  0.818182   
35    0.25     0.4  0.500000  0.571429   0.625  0.666667      0.7  0.727273   
47    0.00     0.2  0.333333  0.428571   0.500  0.555556      0.6  0.636364   

     rest_12   rest_13   rest_14  off_avg_5  off_obp_5  off_slg_5  def_avg_5  \
1   1.000000  1.000000  1.000000   0.240741   0.309392   0.376543   0.229299   
15  0.916667  0.923077  0.928571   0.240741   0.309392   0.376543   0.229299   
25  0.833333  0.846154  0.857143   0.240741   0.309392   0.376543   0.229299   
35  0.750000  0.769231  0.785714   0.240741   0.309392   0.376543   0.229299   
47  0.666667  0.692308  0.714286   0.240741   0.309392   0.376543   0.229299   

    def_whip_5  def_era_5  off_avg_10  off_obp_10  off_slg_10  def_avg_10  \
1     1.031847   1.719745    0.218659    0.281915    0.349854    0.238938   
15    1.031847   1.719745    0.218659    0.281915    0.349854    0.238938   
25    1.031847   1.719745    0.218659    0.281915    0.349854    0.238938   
35    1.031847   1.719745    0.218659    0.281915    0.349854    0.238938   
47    1.031847   1.719745    0.218659    0.281915    0.349854    0.238938   

    def_whip_10  def_era_10  off_avg_20  off_obp_20  off_slg_20  def_avg_20  \
1      1.132743    2.867257    0.231988    0.299346    0.364553    0.254438   
15     1.132743    2.867257    0.231988    0.299346    0.364553    0.254438   
25     1.132743    2.867257    0.231988    0.299346    0.364553    0.254438   
35     1.132743    2.867257    0.231988    0.299346    0.364553    0.254438   
47     1.132743    2.867257    0.231988    0.299346    0.364553    0.254438   

    def_whip_20  def_era_20  off_avg_30  off_obp_30  off_slg_30  def_avg_30  \
1      1.122781    2.556213    0.233918    0.300088    0.362573    0.263054   
15     1.122781    2.556213    0.233918    0.300088    0.362573    0.263054   
25     1.122781    2.556213    0.233918    0.300088    0.362573    0.263054   
35     1.122781    2.556213    0.233918    0.300088    0.362573    0.263054   
47     1.122781    2.556213    0.233918    0.300088    0.362573    0.263054   

    def_whip_30  def_era_30  off_avg_50  off_obp_50  off_slg_50  def_avg_50  \
1      1.155665    2.793103    0.252444    0.314525    0.387579    0.258348   
15     1.155665    2.793103    0.252444    0.314525    0.387579    0.258348   
25     1.155665    2.793103    0.252444    0.314525    0.387579    0.258348   
35     1.155665    2.793103    0.252444    0.314525    0.387579    0.258348   
47     1.155665    2.793103    0.252444    0.314525    0.387579    0.258348   

    def_whip_50  def_era_50  off_avg_100  off_obp_100  off_slg_100  \
1      1.096661    2.862917     0.248109      0.31532     0.385108   
15     1.096661    2.862917     0.248109      0.31532     0.385108   
25     1.096

# game log features and labels

In [177]:
# drop columns from gl that aren't features or labels
gl_fl_cols = [
    'game_id',
    'date',
    'season',
    'visitor_id',
    'home_id',
    'visitor_L',
    'home_L',
    'date_index',
    'RD',
    'W',
    'interleague'
]

gl_features_labels = gl[gl_fl_cols].copy()

gl_features_labels.head()

game_id       date  season  visitor_id  home_id visitor_L home_L  \
0        0 1990-04-09    1990           5        2        AL     AL   
1        1 1990-04-09    1990          11        0        AL     AL   
2        2 1990-04-09    1990          23        3        AL     AL   
3        3 1990-04-09    1990           8       10        AL     AL   
4        4 1990-04-09    1990          14       13        AL     AL   

   date_index  RD  W  interleague  
0         1.0   3  1            0  
1         1.0  -3  0            0  
2         1.0   1  1            0  
3         1.0   5  1            0  
4         1.0   2  1            0

# merge

In [178]:
# merge home team features
home_col_names = {c: 'home_' + c for c in tl_features.columns if c != 'game_id'}

features = pd.merge(
    gl_features_labels,
    tl_features,
    how='left',
    left_on=['game_id', 'home_id'],
    right_on=['game_id', 'team_id']
).drop(columns=['team_id']).rename(columns=home_col_names)

# merge visiting team features
visitor_col_names = {c: 'visitor_' + c for c in tl_features.columns if c != 'game_id'}

features = pd.merge(
    features,
    tl_features,
    how='left',
    left_on=['game_id', 'visitor_id'],
    right_on=['game_id', 'team_id']
).drop(columns=['team_id']).rename(columns=visitor_col_names)

features.head()

game_id       date  season  visitor_id  home_id visitor_L home_L  \
0        0 1990-04-09    1990           5        2        AL     AL   
1        1 1990-04-09    1990          11        0        AL     AL   
2        2 1990-04-09    1990          23        3        AL     AL   
3        3 1990-04-09    1990           8       10        AL     AL   
4        4 1990-04-09    1990          14       13        AL     AL   

   date_index  RD  W  interleague  home_RD_10  home_RD_20  home_RD_30  \
0         1.0   3  1            0        -2.3        0.20    0.000000   
1         1.0  -3  0            0        -0.7       -0.45   -0.933333   
2         1.0   1  1            0        -0.2        1.00    0.666667   
3         1.0   5  1            0         2.0        1.30    1.400000   
4         1.0   2  1            0        -1.2        0.15   -0.133333   

   home_RD_50  home_RD_100  home_win_pct_10  home_win_pct_20  home_win_pct_30  \
0       -0.04        -0.36             0.60            0.525         0.550000   
1       -0.42        -0.50             0.45            0.425         0.516667   
2        0.68         0.38             0.65            0.625         0.600000   
3        1.24         1.11             0.75            0.700         0.666667   
4       -1.22        -0.40             0.55            0.425         0.433333   

   home_win_pct_50  home_win_pct_100  home_rest_1  home_rest_2  home_rest_3  \
0             0.52             0.555          1.0          1.0          1.0   
1             0.48             0.505          1.0          1.0          1.0   
2             0.59             0.555          1.0          1.0          1.0   
3             0.63             0.625          1.0          1.0          1.0   
4             0.50             0.540          1.0          1.0          1.0   

   home_rest_4  home_rest_5  home_rest_6  home_rest_7  home_rest_8  \
0          1.0          1.0          1.0          1.0          1.0   
1          1.0          1.0          1.0          1.0          1.0   
2          1.0          1.0          1.0          1.0          1.0   
3          1.0          1.0          1.0          1.0          1.0   
4          1.0          1.0          1.0          1.0          1.0   

   home_rest_9  home_rest_10  home_rest_11  home_rest_12  home_rest_13  \
0          1.0           1.0           1.0           1.0           1.0   
1          1.0           1.0           1.0           1.0           1.0   
2          1.0           1.0           1.0           1.0           1.0   
3          1.0           1.0           1.0           1.0           1.0   
4          1.0           1.0           1.0           1.0           1.0   

   home_rest_14  home_off_avg_5  home_off_obp_5  home_off_slg_5  \
0           1.0        0.322034        0.396040        0.446328   
1           1.0        0.240741        0.309392        0.376543   
2           1.0        0.253165        0.338798        0.373418   
3           1.0        0.327778        0.389163        0.511111   
4           1.0        0.208861        0.292135        0.360759   

   home_def_avg_5  home_def_whip_5  home_def_era_5  home_off_avg_10  \
0        0.257310         1.140351        3.157895         0.253049   
1        0.229299         1.031847        1.719745         0.218659   
2        0.222222         0.833333        2.500000         0.239264   
3        0.244048         0.910714        2.892857         0.263006   
4        0.232258         1.122581        3.135484         0.233129   

   home_off_obp_10  home_off_slg_10  home_def_avg_10  home_def_whip_10  \
0         0.338667         0.344512         0.284091          1.159091   
1         0.281915         0.349854         0.238938          1.132743   
2         0.307692         0.340491         0.226415          0.915094   
3         0.344304         0.413295         0.230994          0.885965   
4         0.307692         0.349693         0.239748          1.173502   

   home_def_era_10  home_off_avg_2

# export

In [179]:
features.to_parquet('/content/drive/My Drive/mlb/features/features.parquet')

# merge travelling distances

In [180]:
features = pd.read_parquet('/content/drive/My Drive/mlb/features/features.parquet')
features

game_id       date  season  visitor_id  home_id visitor_L home_L  \
0            0 1990-04-09    1990           5        2        AL     AL   
1            1 1990-04-09    1990          11        0        AL     AL   
2            2 1990-04-09    1990          23        3        AL     AL   
3            3 1990-04-09    1990           8       10        AL     AL   
4            4 1990-04-09    1990          14       13        AL     AL   
...        ...        ...     ...         ...      ...       ...    ...   
73131    73131 2021-10-03    2021          23       20        NL     NL   
73132    73132 2021-10-03    2021          19       15        NL     NL   
73133    73133 2021-10-03    2021           4       13        AL     AL   
73134    73134 2021-10-03    2021          17       28        NL     NL   
73135    73135 2021-10-03    2021           1       14        AL     AL   

       date_index  RD  W  interleague  home_RD_10  home_RD_20  home_RD_30  \
0             1.0   3  1            0        -2.3        0.20    0.000000   
1             1.0  -3  0            0        -0.7       -0.45   -0.933333   
2             1.0   1  1            0        -0.2        1.00    0.666667   
3             1.0   5  1            0         2.0        1.30    1.400000   
4             1.0   2  1            0        -1.2        0.15   -0.133333   
...           ...  .. ..          ...         ...         ...         ...   
73131      5583.0   7  1            0         1.2        2.00    1.766667   
73132      5583.0   1  1            0        -0.2       -0.95   -1.266667   
73133      5583.0  -6  0            0        -0.3       -1.80   -1.133333   
73134      5583.0  -1  0            0         2.0        2.85    1.900000   
73135      5583.0   8  1            0        -0.2        1.20    1.966667   

       home_RD_50  home_RD_100  home_win_pct_10  home_win_pct_20  \
0           -0.04        -0.36             0.60            0.525   
1           -0.42        -0.50             0.45            0.425   
2            0.68         0.38             0.65            0.625   
3            1.24         1.11             0.75            0.700   
4           -1.22        -0.40             0.55            0.425   
...           ...          ...              ...              ...   
73131        1.94         1.76             0.85            0.750   
73132       -0.86        -1.52             0.30            0.275   
73133       -1.32        -1.50             0.45            0.425   
73134        1.60         0.62             0.90            0.700   
73135        0.96         1.44             0.60            0.650   

       home_win_pct_30  home_win_pct_50  home_win_pct_100  home_rest_1  \
0             0.550000             0.52             0.555          1.0   
1             0.516667             0.48             0.505          1.0   
2             0.600000             0.59             0.555          1.0   
3             0.666667             0.63             0.625          1.0   
4             0.433333             0.50             0.540          1.0   
...                ...              ...               ...          ...   
73131         0.733333             0.69             0.670          0.0   
73132         0.333333             0.31             0.335          0.0   
73133         0.400000             0.36             0.355          0.0   
73134         0.650000             0.59             0.550          0.0   
73135         0.616667             0.59             0.560          0.0   

       home_rest_2  home_rest_3  home_rest_4  home_rest_5  home_rest_6  \
0              1.0          1.0          1.0          1.0     1.000000   
1              1.0          1.0          1.0          1.0     1.000000   
2              1.0          1.0          1.0          1.0     1.000000   
3              1.0          1.0          1.0          1.0     1.000000   
4              1.0          1.0          1.0          1.0     1.000000   
...            ...          ...

In [195]:
features = pd.merge(
    features,
    gl[['game_id', 'park_ID']],
    how='left',
    on='game_id'
)
features

game_id       date  season  visitor_id  home_id visitor_L home_L  \
0            0 1990-04-09    1990           5        2        AL     AL   
1            1 1990-04-09    1990          11        0        AL     AL   
2            2 1990-04-09    1990          23        3        AL     AL   
3            3 1990-04-09    1990           8       10        AL     AL   
4            4 1990-04-09    1990          14       13        AL     AL   
...        ...        ...     ...         ...      ...       ...    ...   
73131    73131 2021-10-03    2021          23       20        NL     NL   
73132    73132 2021-10-03    2021          19       15        NL     NL   
73133    73133 2021-10-03    2021           4       13        AL     AL   
73134    73134 2021-10-03    2021          17       28        NL     NL   
73135    73135 2021-10-03    2021           1       14        AL     AL   

       date_index  RD  W  interleague  home_RD_10  home_RD_20  home_RD_30  \
0             1.0   3  1            0        -2.3        0.20    0.000000   
1             1.0  -3  0            0        -0.7       -0.45   -0.933333   
2             1.0   1  1            0        -0.2        1.00    0.666667   
3             1.0   5  1            0         2.0        1.30    1.400000   
4             1.0   2  1            0        -1.2        0.15   -0.133333   
...           ...  .. ..          ...         ...         ...         ...   
73131      5583.0   7  1            0         1.2        2.00    1.766667   
73132      5583.0   1  1            0        -0.2       -0.95   -1.266667   
73133      5583.0  -6  0            0        -0.3       -1.80   -1.133333   
73134      5583.0  -1  0            0         2.0        2.85    1.900000   
73135      5583.0   8  1            0        -0.2        1.20    1.966667   

       home_RD_50  home_RD_100  home_win_pct_10  home_win_pct_20  \
0           -0.04        -0.36             0.60            0.525   
1           -0.42        -0.50             0.45            0.425   
2            0.68         0.38             0.65            0.625   
3            1.24         1.11             0.75            0.700   
4           -1.22        -0.40             0.55            0.425   
...           ...          ...              ...              ...   
73131        1.94         1.76             0.85            0.750   
73132       -0.86        -1.52             0.30            0.275   
73133       -1.32        -1.50             0.45            0.425   
73134        1.60         0.62             0.90            0.700   
73135        0.96         1.44             0.60            0.650   

       home_win_pct_30  home_win_pct_50  home_win_pct_100  home_rest_1  \
0             0.550000             0.52             0.555          1.0   
1             0.516667             0.48             0.505          1.0   
2             0.600000             0.59             0.555          1.0   
3             0.666667             0.63             0.625          1.0   
4             0.433333             0.50             0.540          1.0   
...                ...              ...               ...          ...   
73131         0.733333             0.69             0.670          0.0   
73132         0.333333             0.31             0.335          0.0   
73133         0.400000             0.36             0.355          0.0   
73134         0.650000             0.59             0.550          0.0   
73135         0.616667             0.59             0.560          0.0   

       home_rest_2  home_rest_3  home_rest_4  home_rest_5  home_rest_6  \
0              1.0          1.0          1.0          1.0     1.000000   
1              1.0          1.0          1.0          1.0     1.000000   
2              1.0          1.0          1.0          1.0     1.000000   
3              1.0          1.0          1.0          1.0     1.000000   
4              1.0          1.0          1.0          1.0     1.000000   
...            ...          ...

In [182]:
home = features[['game_id', 'home_id', 'park_ID', 'date']].rename(columns={'home_id': 'team_id'})
home['home'] = 1
visitor = features[['game_id', 'visitor_id', 'park_ID', 'date']].rename(columns={'visitor_id': 'team_id'})
visitor['home'] = 0

teams = pd.concat([home, visitor], axis=0).sort_values(by='game_id', axis=0)
teams

game_id  team_id park_ID       date  home
0            0        2   BOS07 1990-04-09     1
0            0        5   BOS07 1990-04-09     0
1            1       11   ANA01 1990-04-09     0
1            1        0   ANA01 1990-04-09     1
2            2        3   CHI10 1990-04-09     1
...        ...      ...     ...        ...   ...
73133    73133       13   ARL03 2021-10-03     1
73134    73134       17   STL10 2021-10-03     0
73134    73134       28   STL10 2021-10-03     1
73135    73135       14   TOR02 2021-10-03     1
73135    73135        1   TOR02 2021-10-03     0

[146272 rows x 5 columns]

In [183]:
teams['prev_park'] = teams.groupby('team_id', as_index=False)[['park_ID']].shift(1)
teams['prev_date'] = teams.groupby('team_id', as_index=False)[['date']].shift(1)
teams.head(50)

game_id  team_id park_ID       date  home prev_park  prev_date
0         0        2   BOS07 1990-04-09     1       NaN        NaT
0         0        5   BOS07 1990-04-09     0       NaN        NaT
1         1       11   ANA01 1990-04-09     0       NaN        NaT
1         1        0   ANA01 1990-04-09     1       NaN        NaT
2         2        3   CHI10 1990-04-09     1       NaN        NaT
2         2       23   CHI10 1990-04-09     0       NaN        NaT
3         3       10   OAK01 1990-04-09     1       NaN        NaT
3         3        8   OAK01 1990-04-09     0       NaN        NaT
4         4       13   ARL01 1990-04-09     1       NaN        NaT
4         4       14   ARL01 1990-04-09     0       NaN        NaT
5         5       18   HOU02 1990-04-09     0       NaN        NaT
5         5        6   HOU02 1990-04-09     1       NaN        NaT
6         6       21   LOS03 1990-04-09     0       NaN        NaT
6         6       20   LOS03 1990-04-09     1       NaN        NaT
7         7       26   NYC17 1990-04-09     0       NaN        NaT
7         7       24   NYC17 1990-04-09     1       NaN        NaT
8         8       29   STL09 1990-04-09     0       NaN        NaT
8         8       28   STL09 1990-04-09     1       NaN        NaT
9         9       29   STL09 1990-04-10     0     STL09 1990-04-09
9         9       28   STL09 1990-04-10     1     STL09 1990-04-09
10       10       20   SAN01 1990-04-10     0     LOS03 1990-04-09
10       10       21   SAN01 1990-04-10     1     LOS03 1990-04-09
11       11       25   CHI11 1990-04-10     0       NaN        NaT
11       11       17   CHI11 1990-04-10     1       NaN        NaT
12       12       14   TOR02 1990-04-10     1     ARL01 1990-04-09
12       12       13   TOR02 1990-04-10     0     ARL01 1990-04-09
13       13        6   HOU02 1990-04-10     1     HOU02 1990-04-09
13       13       18   HOU02 1990-04-10     0     HOU02 1990-04-09
14       14       23   MIL05 1990-04-10     1     CHI10 1990-04-09
14       14        3   MIL05 1990-04-10     0     CHI10 1990-04-09
15       15       11   ANA01 1990-04-10     0     ANA01 1990-04-09
15       15        0   ANA01 1990-04-10     1     ANA01 1990-04-09
16       16        2   BOS07 1990-04-10     1     BOS07 1990-04-09
16       16        5   BOS07 1990-04-10     0     BOS07 1990-04-09
17       17       10   OAK01 1990-04-10     1     OAK01 1990-04-09
17       17        8   OAK01 1990-04-10     0     OAK01 1990-04-09
18       18       28   STL09 1990-04-11     1     STL09 1990-04-10
18       18       29   STL09 1990-04-11     0     STL09 1990-04-10
19       19       21   SAN01 1990-04-11     1     SAN01 1990-04-10
19       19       20   SAN01 1990-04-11     0     SAN01 1990-04-10
20       20       24   NYC17 1990-04-11     1     NYC17 1990-04-09
20       20       26   NYC17 1990-04-11     0     NYC17 1990-04-09
21       21       18   HOU02 1990-04-11     0     HOU02 1990-04-10
21       21        6   HOU02 1990-04-11     1     HOU02 1990-04-10
22       22       16   ATL01 1990-04-11     1       NaN        NaT
22       22       27   ATL01 1990-04-11     0       NaN        NaT
23       23       13   TOR02 1990-04-11     0     TOR02 1990-04-10
23       23       14   TOR02 1990-04-11     1     TOR02 1990-04-10
24       24        7   KAN06 1990-04-11     1       NaN        NaT
24       24        1   KAN06 1990-04-11     0       NaN        NaT

In [184]:
teams['date_diff'] = teams['date'] - teams['prev_date']
teams

game_id  team_id park_ID       date  home prev_park  prev_date  \
0            0        2   BOS07 1990-04-09     1       NaN        NaT   
0            0        5   BOS07 1990-04-09     0       NaN        NaT   
1            1       11   ANA01 1990-04-09     0       NaN        NaT   
1            1        0   ANA01 1990-04-09     1       NaN        NaT   
2            2        3   CHI10 1990-04-09     1       NaN        NaT   
...        ...      ...     ...        ...   ...       ...        ...   
73133    73133       13   ARL03 2021-10-03     1     ARL03 2021-10-02   
73134    73134       17   STL10 2021-10-03     0     STL10 2021-10-02   
73134    73134       28   STL10 2021-10-03     1     STL10 2021-10-02   
73135    73135       14   TOR02 2021-10-03     1     TOR02 2021-10-02   
73135    73135        1   TOR02 2021-10-03     0     TOR02 2021-10-02   

      date_diff  
0           NaT  
0           NaT  
1           NaT  
1           NaT  
2           NaT  
...         ...  
73133    1 days  
73134    1 days  
73134    1 days  
73135    1 days  
73135    1 days  

[146272 rows x 8 columns]

In [185]:
team_distances = pd.merge(
    teams,
    dist_df,
    how='left',
    left_on=['park_ID', 'prev_park'],
    right_on=['PARKID_x', 'PARKID_y']
).drop(columns=['PARKID_x', 'PARKID_y'])

In [186]:
team_distances['geodesic distance (km)'] = team_distances['geodesic distance (km)'].fillna(0)

In [187]:
team_distances['geodesic distance (km)'].plot.hist()

In [190]:
team_distances['date_diff'] = team_distances['date_diff'].dt.days

In [191]:
features = pd.merge(
    features,
    team_distances.loc[team_distances['home'] == 0, ['game_id', 'team_id', 'date_diff', 'geodesic distance (km)']],
    how='left',
    left_on=['game_id', 'visitor_id'],
    right_on=['game_id', 'team_id']
).drop(columns=['team_id']).rename(columns={'date_diff': 'visitor_travel_days', 'geodesic distance (km)': 'visitor_travel_distance'})

In [192]:
features

game_id       date  season  visitor_id  home_id visitor_L home_L  \
0            0 1990-04-09    1990           5        2        AL     AL   
1            1 1990-04-09    1990          11        0        AL     AL   
2            2 1990-04-09    1990          23        3        AL     AL   
3            3 1990-04-09    1990           8       10        AL     AL   
4            4 1990-04-09    1990          14       13        AL     AL   
...        ...        ...     ...         ...      ...       ...    ...   
73131    73131 2021-10-03    2021          23       20        NL     NL   
73132    73132 2021-10-03    2021          19       15        NL     NL   
73133    73133 2021-10-03    2021           4       13        AL     AL   
73134    73134 2021-10-03    2021          17       28        NL     NL   
73135    73135 2021-10-03    2021           1       14        AL     AL   

       date_index  RD  W  interleague  home_RD_10  home_RD_20  home_RD_30  \
0             1.0   3  1            0        -2.3        0.20    0.000000   
1             1.0  -3  0            0        -0.7       -0.45   -0.933333   
2             1.0   1  1            0        -0.2        1.00    0.666667   
3             1.0   5  1            0         2.0        1.30    1.400000   
4             1.0   2  1            0        -1.2        0.15   -0.133333   
...           ...  .. ..          ...         ...         ...         ...   
73131      5583.0   7  1            0         1.2        2.00    1.766667   
73132      5583.0   1  1            0        -0.2       -0.95   -1.266667   
73133      5583.0  -6  0            0        -0.3       -1.80   -1.133333   
73134      5583.0  -1  0            0         2.0        2.85    1.900000   
73135      5583.0   8  1            0        -0.2        1.20    1.966667   

       home_RD_50  home_RD_100  home_win_pct_10  home_win_pct_20  \
0           -0.04        -0.36             0.60            0.525   
1           -0.42        -0.50             0.45            0.425   
2            0.68         0.38             0.65            0.625   
3            1.24         1.11             0.75            0.700   
4           -1.22        -0.40             0.55            0.425   
...           ...          ...              ...              ...   
73131        1.94         1.76             0.85            0.750   
73132       -0.86        -1.52             0.30            0.275   
73133       -1.32        -1.50             0.45            0.425   
73134        1.60         0.62             0.90            0.700   
73135        0.96         1.44             0.60            0.650   

       home_win_pct_30  home_win_pct_50  home_win_pct_100  home_rest_1  \
0             0.550000             0.52             0.555          1.0   
1             0.516667             0.48             0.505          1.0   
2             0.600000             0.59             0.555          1.0   
3             0.666667             0.63             0.625          1.0   
4             0.433333             0.50             0.540          1.0   
...                ...              ...               ...          ...   
73131         0.733333             0.69             0.670          0.0   
73132         0.333333             0.31             0.335          0.0   
73133         0.400000             0.36             0.355          0.0   
73134         0.650000             0.59             0.550          0.0   
73135         0.616667             0.59             0.560          0.0   

       home_rest_2  home_rest_3  home_rest_4  home_rest_5  home_rest_6  \
0              1.0          1.0          1.0          1.0     1.000000   
1              1.0          1.0          1.0          1.0     1.000000   
2              1.0          1.0          1.0          1.0     1.000000   
3              1.0          1.0          1.0          1.0     1.000000   
4              1.0          1.0          1.0          1.0     1.000000   
...            ...          ...

In [196]:
features = pd.merge(
    features,
    team_distances.loc[team_distances['home'] == 1, ['game_id', 'team_id', 'date_diff', 'geodesic distance (km)']],
    how='left',
    left_on=['game_id', 'home_id'],
    right_on=['game_id', 'team_id']
).drop(columns=['park_ID', 'team_id']).rename(columns={'date_diff': 'home_travel_days', 'geodesic distance (km)': 'home_travel_distance'})

In [197]:
features

game_id       date  season  visitor_id  home_id visitor_L home_L  \
0            0 1990-04-09    1990           5        2        AL     AL   
1            1 1990-04-09    1990          11        0        AL     AL   
2            2 1990-04-09    1990          23        3        AL     AL   
3            3 1990-04-09    1990           8       10        AL     AL   
4            4 1990-04-09    1990          14       13        AL     AL   
...        ...        ...     ...         ...      ...       ...    ...   
73131    73131 2021-10-03    2021          23       20        NL     NL   
73132    73132 2021-10-03    2021          19       15        NL     NL   
73133    73133 2021-10-03    2021           4       13        AL     AL   
73134    73134 2021-10-03    2021          17       28        NL     NL   
73135    73135 2021-10-03    2021           1       14        AL     AL   

       date_index  RD  W  interleague  home_RD_10  home_RD_20  home_RD_30  \
0             1.0   3  1            0        -2.3        0.20    0.000000   
1             1.0  -3  0            0        -0.7       -0.45   -0.933333   
2             1.0   1  1            0        -0.2        1.00    0.666667   
3             1.0   5  1            0         2.0        1.30    1.400000   
4             1.0   2  1            0        -1.2        0.15   -0.133333   
...           ...  .. ..          ...         ...         ...         ...   
73131      5583.0   7  1            0         1.2        2.00    1.766667   
73132      5583.0   1  1            0        -0.2       -0.95   -1.266667   
73133      5583.0  -6  0            0        -0.3       -1.80   -1.133333   
73134      5583.0  -1  0            0         2.0        2.85    1.900000   
73135      5583.0   8  1            0        -0.2        1.20    1.966667   

       home_RD_50  home_RD_100  home_win_pct_10  home_win_pct_20  \
0           -0.04        -0.36             0.60            0.525   
1           -0.42        -0.50             0.45            0.425   
2            0.68         0.38             0.65            0.625   
3            1.24         1.11             0.75            0.700   
4           -1.22        -0.40             0.55            0.425   
...           ...          ...              ...              ...   
73131        1.94         1.76             0.85            0.750   
73132       -0.86        -1.52             0.30            0.275   
73133       -1.32        -1.50             0.45            0.425   
73134        1.60         0.62             0.90            0.700   
73135        0.96         1.44             0.60            0.650   

       home_win_pct_30  home_win_pct_50  home_win_pct_100  home_rest_1  \
0             0.550000             0.52             0.555          1.0   
1             0.516667             0.48             0.505          1.0   
2             0.600000             0.59             0.555          1.0   
3             0.666667             0.63             0.625          1.0   
4             0.433333             0.50             0.540          1.0   
...                ...              ...               ...          ...   
73131         0.733333             0.69             0.670          0.0   
73132         0.333333             0.31             0.335          0.0   
73133         0.400000             0.36             0.355          0.0   
73134         0.650000             0.59             0.550          0.0   
73135         0.616667             0.59             0.560          0.0   

       home_rest_2  home_rest_3  home_rest_4  home_rest_5  home_rest_6  \
0              1.0          1.0          1.0          1.0     1.000000   
1              1.0          1.0          1.0          1.0     1.000000   
2              1.0          1.0          1.0          1.0     1.000000   
3              1.0          1.0          1.0          1.0     1.000000   
4              1.0          1.0          1.0          1.0     1.000000   
...            ...          ...

In [198]:
features = features.rename(columns={'visitor_travel_days': 'visitor_days_since_previous_game',
                                    'home_travel_days': 'home_days_since_previous_game'})

In [199]:
features

game_id       date  season  visitor_id  home_id visitor_L home_L  \
0            0 1990-04-09    1990           5        2        AL     AL   
1            1 1990-04-09    1990          11        0        AL     AL   
2            2 1990-04-09    1990          23        3        AL     AL   
3            3 1990-04-09    1990           8       10        AL     AL   
4            4 1990-04-09    1990          14       13        AL     AL   
...        ...        ...     ...         ...      ...       ...    ...   
73131    73131 2021-10-03    2021          23       20        NL     NL   
73132    73132 2021-10-03    2021          19       15        NL     NL   
73133    73133 2021-10-03    2021           4       13        AL     AL   
73134    73134 2021-10-03    2021          17       28        NL     NL   
73135    73135 2021-10-03    2021           1       14        AL     AL   

       date_index  RD  W  interleague  home_RD_10  home_RD_20  home_RD_30  \
0             1.0   3  1            0        -2.3        0.20    0.000000   
1             1.0  -3  0            0        -0.7       -0.45   -0.933333   
2             1.0   1  1            0        -0.2        1.00    0.666667   
3             1.0   5  1            0         2.0        1.30    1.400000   
4             1.0   2  1            0        -1.2        0.15   -0.133333   
...           ...  .. ..          ...         ...         ...         ...   
73131      5583.0   7  1            0         1.2        2.00    1.766667   
73132      5583.0   1  1            0        -0.2       -0.95   -1.266667   
73133      5583.0  -6  0            0        -0.3       -1.80   -1.133333   
73134      5583.0  -1  0            0         2.0        2.85    1.900000   
73135      5583.0   8  1            0        -0.2        1.20    1.966667   

       home_RD_50  home_RD_100  home_win_pct_10  home_win_pct_20  \
0           -0.04        -0.36             0.60            0.525   
1           -0.42        -0.50             0.45            0.425   
2            0.68         0.38             0.65            0.625   
3            1.24         1.11             0.75            0.700   
4           -1.22        -0.40             0.55            0.425   
...           ...          ...              ...              ...   
73131        1.94         1.76             0.85            0.750   
73132       -0.86        -1.52             0.30            0.275   
73133       -1.32        -1.50             0.45            0.425   
73134        1.60         0.62             0.90            0.700   
73135        0.96         1.44             0.60            0.650   

       home_win_pct_30  home_win_pct_50  home_win_pct_100  home_rest_1  \
0             0.550000             0.52             0.555          1.0   
1             0.516667             0.48             0.505          1.0   
2             0.600000             0.59             0.555          1.0   
3             0.666667             0.63             0.625          1.0   
4             0.433333             0.50             0.540          1.0   
...                ...              ...               ...          ...   
73131         0.733333             0.69             0.670          0.0   
73132         0.333333             0.31             0.335          0.0   
73133         0.400000             0.36             0.355          0.0   
73134         0.650000             0.59             0.550          0.0   
73135         0.616667             0.59             0.560          0.0   

       home_rest_2  home_rest_3  home_rest_4  home_rest_5  home_rest_6  \
0              1.0          1.0          1.0          1.0     1.000000   
1              1.0          1.0          1.0          1.0     1.000000   
2              1.0          1.0          1.0          1.0     1.000000   
3              1.0          1.0          1.0          1.0     1.000000   
4              1.0          1.0          1.0          1.0     1.000000   
...            ...          ...

In [200]:
features.to_parquet('/content/drive/My Drive/mlb/features/features.parquet')